### 1.c - Auditoria no Código Postal (Postal Code)

In [35]:
import xml.etree.cElementTree as ET
from collections import defaultdict
import re
import pprint

OSMFILE = "dados/jundiai_e_regiao_map_zen.osm"

accept_pc1 = re.compile(r'(.....?-...)')
accept_pc2 = re.compile(r'(........)')
problemchars = re.compile(r'([=\+/&<>;\'"\?%#$@\,\.\t\r\n]|[A-z])')

postal_c = None 


def audit_characters_postal_code(element):
    classificated = False
    result = False

    # INCOMPLETO
    if len(element.replace("-","")) < 8:
        result = False
        classificated = True        

    # PROBLEMCHAR
    if classificated == False:
        if problemchars.search(element) is not None:
            classificated = True        
        
    # CORRETO 2
    if not classificated:
        if accept_pc1.search(element) is not None:        
            result = True
            classificated = True   
            
    # CORRETO 1
    if not classificated:
        if accept_pc2.search(element) is not None:        
            result = False
            classificated = True   
    
    # OUTROS
    if not classificated:
        print(element, ' = classif.: OTHER')                  
    
    return result

def is_postal_code(elem):
    return ((elem.attrib['k'] == "addr:postcode") | (elem.attrib['k'] == "postal_code"))

def print_sorted_dict(d):
    keys = d.keys()
    keys = sorted(keys, key=lambda s: s.lower())
    for k in keys:
        v = d[k]
        print("%s: %d" % (k, v))

def audit(osmfile):
    osm_file = open(osmfile, "r", encoding="utf-8")
    postal_codes = defaultdict(set)
    
    for event, elem in ET.iterparse(osm_file, events=("start",)):
        #print('\n',elem.tag, elem.attrib)    
       
        if elem.tag == "node" or elem.tag == "way":
            #print('\n',elem.tag, elem.attrib)

            for tag in elem.iter("tag"):                
                if is_postal_code(tag): 
                    if audit_characters_postal_code(tag.attrib['v']) == False:
                        postal_c = tag.attrib['v']
                        if postal_codes[postal_c]== set():
                            postal_codes[postal_c] = 0

                        postal_codes[postal_c]+= 1
                        
                            
    print_sorted_dict(postal_codes)                
    osm_file.close()


def test():
    # Auditar para encontrar códigos postais (CEPs) fora do padrão
    st_types = audit(OSMFILE)
            

if __name__ == '__main__':
    test()
    

07600000: 129
12940700: 1
12942540: 1
12942655: 10
12943000: 2
12943310: 7
12943320: 13
12943330: 31
12943340: 38
12943350: 16
12943370: 13
12943380: 3
12943500: 12
12947452: 8
13203280: 1
13221550: 1
13240000: 6
3221-390: 1
